# 2. Given the following confusion matrix, evaluate (by hand) the model's performance.

|               | pred dog   | pred cat   |
|:------------  |-----------:|-----------:|
| actual dog    |         46 |         7  |
| actual cat    |         13 |         34 |


2a. In the context of this problem, what is a false positive?

2b. In the context of this problem, what is a false negative?

2c. How would you describe this model?

# 3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.
Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.: https://ds.codeup.com/data/c3.csv

Use the predictions dataset and pandas to help answer the following questions:

#### 3a. An internal team wants to investigate the cause of the manufacturing defects. 
They tell you that they want to identify as many of the ducks that have a defect as possible. 
- Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

#### 3b. Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. 
The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. 

They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. 
- Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

# 4. You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).

At Gives You Paws, anyone can upload pictures of their cats or dogs. 
The photos are then put through a two step process. 
- First an automated algorithm tags pictures as either a cat or a dog **(Phase I)**. 
- Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users **(Phase II)**.

Several models have already been developed with the data, and you can find their results here.: https://ds.codeup.com/data/gives_you_paws.csv

Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions: